# Paso 1: Configuración e Instalaciones
Esta sección se encarga de preparar el entorno de Colab, montando Google Drive para la gestión de archivos y asegurándose de que todas las librerías necesarias estén instaladas.

## SECCIÓN 1: CONFIGURACIÓN E INSTALACIONES

## Montar Google Drive para guardar/cargar archivos fácilmente.
Esto es útil para guardar el modelo entrenado y luego descargarlo a tu máquina local.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Instalar librerías necesarias.
'tensorflow-hub' para cargar el modelo YAMNet.
'librosa' para el procesamiento de audio (carga, resampleo, aumento de datos).
'soundfile' es útil para manejar archivos de audio.
'zipfile' se usará para descomprimir el dataset RAVDESS.
'scikit-learn' para dividir los datos y escalar características.


In [2]:
print("Instalando librerías...")
!pip install tensorflow-hub librosa soundfile numpy pandas scikit-learn tensorflow
print("Librerías instaladas.")

# Importar todas las librerías que usaremos a lo largo del cuaderno.
import tensorflow as tf
import tensorflow_hub as hub
import librosa
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout # Dropout es crucial para prevenir el sobreajuste
from tensorflow.keras.optimizers import Adam
import zipfile

print("Librerías importadas.")


Instalando librerías...
Librerías instaladas.
Librerías importadas.


# Paso 2: Descarga y Organización del Dataset RAVDESS
Aquí descargaremos el dataset RAVDESS, un conjunto de datos de emociones en el habla. Luego, lo organizaremos y mapearemos las emociones de RAVDESS a nuestras dos categorías de "predisposición" (1) o "no predisposición" (0) de manera heurística. Recuerda que este mapeo es una interpretación y puedes ajustarlo si lo consideras necesario.

## SECCIÓN 2: DESCARGA Y ORGANIZACIÓN DEL DATASET RAVDESS

URL para descargar RAVDESS. Es posible que tengas que descargarlo manualmente
desde el sitio web oficial (buscar "RAVDESS dataset") y luego subirlo a Colab o a tu Drive.



In [4]:
# Si ya lo tienes en tu Google Drive, ajusta la ruta 'RAVDESS_ZIP_PATH' a tu ubicación específica.
RAVDESS_ZIP_PATH = '/content/drive/MyDrive/Audio_Speech_Actors_01-24.zip' # RUTA DE EJEMPLO EN DRIVE
# Si lo subiste directamente a la sesión de Colab, podría ser 'RAVDESS_Speech_Song_Actors_01-24.zip'

# Directorio donde se extraerán los archivos del dataset
DATASET_DIR = '/content/ravdess_extracted'
os.makedirs(DATASET_DIR, exist_ok=True)

print(f"Descomprimiendo {RAVDESS_ZIP_PATH} en {DATASET_DIR}...")
try:
    with zipfile.ZipFile(RAVDESS_ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(DATASET_DIR)
    print("Dataset RAVDESS descomprimido exitosamente.")
except FileNotFoundError:
    print(f"ERROR: El archivo ZIP de RAVDESS no se encontró en '{RAVDESS_ZIP_PATH}'.")
    print("Por favor, asegúrate de que el archivo ZIP esté en la ruta especificada (por ejemplo, en tu Google Drive).")
    raise FileNotFoundError("RAVDESS ZIP file not found. Please upload or adjust path.")
except Exception as e:
    print(f"ERROR al descomprimir el dataset: {e}")
    raise

# Estructura de nombres de archivo de RAVDESS: Modality-VocalChannel-Emotion-Intensity-Statement-Repetition-Actor.file_extension
# El tercer número es el ID de la emoción:
# 01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised.

# Mapeo de IDs de emoción de RAVDESS a nombres de emoción legibles.
emotion_id_to_name = {
    '01': 'neutral', '02': 'calm', '03': 'happy', '04': 'sad',
    '05': 'angry', '06': 'fearful', '07': 'disgust', '08': 'surprised'
}

# --- Mapeo HEURÍSTICO de emociones a "Predispuesto" (1) o "No Predispuesto" (0) ---
# Este es el paso crucial donde interpretamos las emociones existentes para tu tarea.
# Ajusta este mapeo según tu comprensión de lo que indica "predisposición".
predisposition_mapping = {
    'neutral': 1,      # Un tono neutral puede indicar apertura o predisposición.
    'calm': 1,         # La calma a menudo se asocia con disposición positiva.
    'happy': 1,        # Felicidad es un fuerte indicador de predisposición.
    'surprised': 1,    # Sorpresa (positiva) puede indicar interés o predisposición.
    'sad': 0,          # Tristeza suele indicar falta de predisposición.
    'angry': 0,        # El enojo claramente indica no predisposición.
    'fearful': 0,      # El miedo indica no predisposición.
    'disgust': 0       # El disgusto indica no predisposición.
}

all_audio_files = []
all_labels = []

print("Procesando archivos de audio y extrayendo etiquetas de predisposición...")
# Recorrer las carpetas de actores (Actor_01, Actor_02, etc.)
for actor_folder in os.listdir(DATASET_DIR):
    actor_path = os.path.join(DATASET_DIR, actor_folder)
    if os.path.isdir(actor_path): # Asegurarse de que es un directorio
        for audio_file in os.listdir(actor_path):
            if audio_file.endswith('.wav'): # Solo procesar archivos de audio WAV
                file_path = os.path.join(actor_path, audio_file)

                # Extraer el ID de la emoción del nombre del archivo (ej: 03-01-03-...)
                emotion_id = audio_file.split('-')[2]

                # Obtener el nombre de la emoción y luego la etiqueta de predisposición
                emotion_name = emotion_id_to_name.get(emotion_id)
                if emotion_name: # Si la emoción es reconocida
                    predisposition_label = predisposition_mapping.get(emotion_name)
                    if predisposition_label is not None: # Si tiene un mapeo a predisposición
                        all_audio_files.append(file_path)
                        all_labels.append(predisposition_label)

# Crear un DataFrame de Pandas para organizar y gestionar los datos de audio y sus etiquetas.
df_audio = pd.DataFrame({'file_path': all_audio_files, 'label': all_labels})
print(f"Total de archivos de audio procesados: {len(df_audio)}")
print("Distribución de etiquetas de predisposición:")
print(df_audio['label'].value_counts()) # Muestra cuántos audios hay de cada clase (0 o 1)


Descomprimiendo /content/drive/MyDrive/Audio_Speech_Actors_01-24.zip en /content/ravdess_extracted...
Dataset RAVDESS descomprimido exitosamente.
Procesando archivos de audio y extrayendo etiquetas de predisposición...
Total de archivos de audio procesados: 1440
Distribución de etiquetas de predisposición:
label
0    768
1    672
Name: count, dtype: int64


# Paso 3: Extracción de Embeddings con YAMNet y Data Augmentation
Esta es la parte central del transfer learning. Cargamos YAMNet (un modelo pre-entrenado por Google) y lo usamos para extraer representaciones numéricas de alto nivel (llamadas embeddings) de cada archivo de audio. También implementamos data augmentation (aumento de datos) para crear variaciones de nuestros audios, lo que ayuda al modelo a generalizar mejor y reduce el sobreajuste.

## SECCIÓN 3: EXTRACCIÓN DE EMBEDDINGS CON YAMNET Y DATA AUGMENTATION

Cargar el modelo YAMNet de TensorFlow Hub.
YAMNet es un modelo pre-entrenado para clasificación de eventos sonoros.
Lo utilizaremos como un potente extractor de características de audio.

In [6]:
print("Cargando modelo YAMNet...")
yamnet_model_handle = 'https://tfhub.dev/google/yamnet/1'
yamnet_model = hub.load(yamnet_model_handle)
print("Modelo YAMNet cargado.")

# Configuración para el preprocesamiento de audio, DEBE COINCIDIR con YAMNet y con tu app final.
SAMPLE_RATE_YAMNET = 16000 # YAMNet fue entrenado con audio a 16kHz
DURATION_SECONDS_YAMNET = 3 # Duración del clip de audio a procesar para el embedding
                            # Cada segmento de audio se padeará/truncará a esta duración.

def preprocess_audio_for_yamnet_with_augmentation(file_path, sr=SAMPLE_RATE_YAMNET, duration=DURATION_SECONDS_YAMNET, augment=False):
    """
    Carga un archivo de audio, lo resamplea, lo pad/trunca, lo normaliza y, opcionalmente,
    aplica técnicas de data augmentation. YAMNet espera la forma de onda de audio (raw waveform).
    """
    try:
        y, current_sr = librosa.load(file_path, sr=sr, mono=True) # Cargar y resamplear

        # Normalizar a -1 a 1 para un mejor procesamiento.
        y = y / np.max(np.abs(y)) if np.max(np.abs(y)) > 0 else y

        # Padear o truncar el audio para asegurar una duración fija.
        target_length = sr * duration
        if len(y) < target_length:
            y = np.pad(y, (0, target_length - len(y)), mode='constant')
        elif len(y) > target_length:
            y = y[:target_length]

        # --- APLICAR DATA AUGMENTATION (solo si 'augment' es True) ---
        if augment:
            # 1. Añadir Ruido Blanco Aleatorio
            # Esto ayuda al modelo a ser más robusto a las variaciones de ruido ambiental.
            noise_amp = 0.005 * np.random.uniform() * np.amax(y)
            y = y + noise_amp * np.random.normal(size=y.shape[0])

            # 2. Cambiar Volumen Ligeramente
            # Simula diferentes volúmenes de grabación.
            y = y * np.random.uniform(low=0.8, high=1.2) # Multiplica por un factor entre 0.8 y 1.2

            # 3. Time Stretching (cambio de velocidad sin cambiar el tono)
            # Esto hace que el modelo sea robusto a variaciones en la velocidad del habla.
            # Asegúrate de que el audio resultante siga teniendo la longitud deseada.
            stretch_rate = np.random.uniform(0.9, 1.1) # Alarga o acorta un 10%
            y = librosa.effects.time_stretch(y, rate=stretch_rate)
            # Volver a padear/truncar si el time stretch cambió la longitud
            if len(y) < target_length:
                y = np.pad(y, (0, target_length - len(y)), mode='constant')
            elif len(y) > target_length:
                y = y[:target_length]

            # 4. Pitch Shifting (cambio de tono)
            # Simula diferentes tonos de voz.
            # n_steps: número de semitonos para cambiar el tono (ej: -2 a 2 semitonos).
            y = librosa.effects.pitch_shift(y, sr=sr, n_steps=np.random.uniform(-2, 2))

            # Normalizar de nuevo después de la aumentación si los cambios fueron significativos
            y = y / np.max(np.abs(y)) if np.max(np.abs(y)) > 0 else y

        return y.astype(np.float32) # YAMNet espera float32

    except Exception as e:
        print(f"Error al preprocesar (con/sin aug) {file_path}: {e}")
        return None

def extract_yamnet_embeddings(audio_waveform, yamnet_model):
    """
    Extrae embeddings de YAMNet de un waveform de audio.
    """
    # YAMNet devuelve scores (probabilidades), embeddings (características) y un log_mel_spectrogram.
    # Nos interesan los 'embeddings' para la clasificación.
    scores, embeddings, spectrogram = yamnet_model(audio_waveform)

    # YAMNet puede devolver una secuencia de embeddings si el audio es largo.
    # Para nuestro clasificador final (una red densa simple), promediamos los embeddings
    # a lo largo del tiempo para obtener un único vector de características por clip.
    if embeddings.shape[0] > 0:
        return np.mean(embeddings.numpy(), axis=0) # Convertir a numpy y promediar
    else:
        return None # Retorna None si no se pudieron extraer embeddings válidos

all_embeddings = []
all_labels = []

print("Extrayendo embeddings de YAMNet (originales y aumentados) de los archivos de audio...")
# Iterar sobre cada archivo de audio en el DataFrame
for index, row in df_audio.iterrows():
    file_path = row['file_path']
    label = row['label']

    # 1. Procesar el audio original (sin aumento)
    waveform_orig = preprocess_audio_for_yamnet_with_augmentation(file_path, augment=False)
    if waveform_orig is not None:
        embedding_orig = extract_yamnet_embeddings(waveform_orig, yamnet_model)
        if embedding_orig is not None:
            all_embeddings.append(embedding_orig)
            all_labels.append(label)

    # 2. Procesar versiones aumentadas del audio
    # Podemos crear múltiples variantes aumentadas por cada archivo original.
    # Esto multiplica efectivamente el tamaño de nuestro dataset de entrenamiento.
    NUM_AUGMENTATIONS_PER_SAMPLE = 2 # Puedes ajustar el número de versiones aumentadas
    for _ in range(NUM_AUGMENTATIONS_PER_SAMPLE):
        waveform_aug = preprocess_audio_for_yamnet_with_augmentation(file_path, augment=True)
        if waveform_aug is not None:
            embedding_aug = extract_yamnet_embeddings(waveform_aug, yamnet_model)
            if embedding_aug is not None:
                all_embeddings.append(embedding_aug)
                all_labels.append(label)

# Convertir las listas a arrays de NumPy para el entrenamiento del modelo.
X_embeddings = np.array(all_embeddings)
y_labels = np.array(all_labels)

# Guardar los embeddings y las etiquetas (opcional, pero útil si la sesión de Colab se desconecta).
OUTPUT_EMBEDDINGS_DIR = '/content/audio_embeddings'
os.makedirs(OUTPUT_EMBEDDINGS_DIR, exist_ok=True)
np.save(os.path.join(OUTPUT_EMBEDDINGS_DIR, 'X_audio_embeddings.npy'), X_embeddings)
np.save(os.path.join(OUTPUT_EMBEDDINGS_DIR, 'y_audio_labels.npy'), y_labels)

print(f"Embeddings extraídos (originales + aumentados). Forma de X: {X_embeddings.shape}, Forma de y: {y_labels.shape}")
print(f"Los embeddings y etiquetas han sido guardados en {OUTPUT_EMBEDDINGS_DIR}.")


Cargando modelo YAMNet...
Modelo YAMNet cargado.
Extrayendo embeddings de YAMNet (originales y aumentados) de los archivos de audio...
Embeddings extraídos (originales + aumentados). Forma de X: (4320, 1024), Forma de y: (4320,)
Los embeddings y etiquetas han sido guardados en /content/audio_embeddings.


# Paso 4: Entrenamiento del Modelo de Clasificación (Fine-tuning)
Aquí construiremos y entrenaremos una pequeña red neuronal densa sobre los embeddings extraídos por YAMNet. Esta red actuará como la "cabeza" de nuestro modelo de transfer learning, adaptando las características generales de YAMNet a nuestra tarea específica de "predisposición". Se han añadido capas Dropout para mejorar la generalización del modelo.

## SECCIÓN 4: ENTRENAMIENTO DEL MODELO DE CLASIFICACIÓN (FINE-TUNING)

Cargar los embeddings y etiquetas (si no se procesaron en la sesión actual).
Esto es útil si reinicias la sesión de Colab y quieres cargar los datos ya extraídos.


In [7]:
try:
    X = np.load(os.path.join(OUTPUT_EMBEDDINGS_DIR, 'X_audio_embeddings.npy'))
    y = np.load(os.path.join(OUTPUT_EMBEDDINGS_DIR, 'y_audio_labels.npy'))
    print("Embeddings y etiquetas cargados de archivos.")
except FileNotFoundError:
    print("ERROR: No se encontraron los archivos de embeddings. Asegúrate de ejecutar la sección anterior.")
    raise

print(f"Dimensiones de X: {X.shape}, Dimensiones de y: {y.shape}")

# Dividir los datos en conjuntos de entrenamiento y prueba.
# test_size=0.2: el 20% de los datos se usarán para probar el modelo.
# random_state=42: asegura que la división sea la misma cada vez para reproducibilidad.
# stratify=y: crucial para mantener la misma proporción de clases (predispuesto/no predispuesto)
# en los conjuntos de entrenamiento y prueba, especialmente si las clases están desequilibradas.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Datos divididos: X_train={X_train.shape}, X_test={X_test.shape}")

# Escalado de características.
# Es importante escalar los datos (normalmente a media 0 y varianza 1) para que las
# redes neuronales converjan mejor y más rápido.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) # Ajusta el escalador y transforma los datos de entrenamiento.
X_test_scaled = scaler.transform(X_test)       # Transforma los datos de prueba usando el escalador ajustado.

print("Características escaladas.")

# --- Diseño del Modelo de Clasificación (Red Neuronal con Capas Dropout) ---
# Este modelo tomará los embeddings de YAMNet (1024 dimensiones) como entrada.
# Añadir capas Dropout es una técnica efectiva para prevenir el sobreajuste.
# Dropout "apaga" aleatoriamente un porcentaje de neuronas durante el entrenamiento,
# forzando a la red a aprender representaciones más robustas y menos dependientes
# de neuronas específicas.
model = Sequential([
    # Capa de entrada: Dense con 256 neuronas, activación 'relu'.
    # input_shape: Debe coincidir con la dimensión de los embeddings de YAMNet (1024).
    Dense(256, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.5), # Primera capa Dropout: apaga el 50% de las neuronas aleatoriamente.
                  # Un valor alto (0.5) es común para las primeras capas densas.

    Dense(128, activation='relu'), # Segunda capa oculta: Dense con 128 neuronas.
    Dropout(0.4), # Segunda capa Dropout: apaga el 40% de las neuronas.
                  # Puedes usar un valor ligeramente menor que la primera.

    # Capa de salida: Dense con 1 neurona y activación 'sigmoid'.
    # 'sigmoid' es estándar para clasificación binaria, ya que emite un valor entre 0 y 1
    # que puede interpretarse como la probabilidad de la clase positiva (predispuesto).
    Dense(1, activation='sigmoid')
])

# Compilación del Modelo.
# optimizer: Algoritmo de optimización (Adam es una excelente opción por defecto).
#            learning_rate (tasa de aprendizaje) es crucial: un valor pequeño (0.0001)
#            es ideal para fine-tuning sobre características pre-entrenadas.
# loss: Función de pérdida (binary_crossentropy es la apropiada para clasificación binaria).
# metrics: Métrica para monitorear durante el entrenamiento y evaluación (accuracy es la precisión).
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

print("\nResumen del Modelo:")
model.summary()

# Entrenamiento del Modelo.
# epochs: Número de veces que el modelo verá todos los datos de entrenamiento.
#         Ajusta esto: más épocas si el modelo aún mejora, menos si ya se sobreajusta.
# batch_size: Número de ejemplos procesados antes de actualizar los pesos del modelo.
# validation_split: Porcentaje de los datos de entrenamiento a usar para validación.
#                   Estos datos no se usan para actualizar los pesos, solo para monitorear
#                   el rendimiento del modelo en datos no vistos durante cada época.
print("\nEntrenando el modelo de audio...")
history = model.fit(X_train_scaled, y_train,
                    epochs=100, # Un buen punto de partida, puedes aumentar/disminuir.
                    batch_size=32,
                    validation_split=0.2, # El 20% de los datos de entrenamiento se usará para validación.
                    verbose=1) # Muestra el progreso del entrenamiento en la salida.

# Evaluación del Modelo.
# Evalúa el rendimiento final del modelo en los datos de prueba (que nunca ha visto).
loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"\nPrecisión final del modelo en datos de prueba: {accuracy:.4f}")

# Guardar el Modelo.
# Guardar el modelo en formato .h5 para poder cargarlo en tu aplicación Streamlit.
# Lo guardamos directamente en tu Google Drive para facilitar la descarga.
MODEL_SAVE_PATH = "/content/drive/MyDrive/audio_emotion_model.h5"
# Asegúrate de que la carpeta de destino en Drive existe (Colab la crea si no).
os.makedirs(os.path.dirname(MODEL_SAVE_PATH), exist_ok=True)
model.save(MODEL_SAVE_PATH)
print(f"Modelo de audio guardado exitosamente en: {MODEL_SAVE_PATH}")
print("Puedes descargarlo desde tu Google Drive y colocarlo en la carpeta 'model/' de tu proyecto Streamlit.")


Embeddings y etiquetas cargados de archivos.
Dimensiones de X: (4320, 1024), Dimensiones de y: (4320,)
Datos divididos: X_train=(3456, 1024), X_test=(864, 1024)
Características escaladas.

Resumen del Modelo:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │       262,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 295,425 (1.13 MB)

 Trainable params: 295,425 (1.13 MB)

 Non-trainable params: 0 (0.00 B)


Entrenando el modelo de audio...
Epoch 1/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/step - accuracy: 0.5323 - loss: 0.8438 - val_accuracy: 0.5535 - val_loss: 0.6831
Epoch 2/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5378 - loss: 0.7929 - val_accuracy: 0.5867 - val_loss: 0.6871
Epoch 3/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5584 - loss: 0.7424 - val_accuracy: 0.5882 - val_loss: 0.6728
Epoch 4/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5810 - loss: 0.7121 - val_accuracy: 0.5795 - val_loss: 0.7125
Epoch 5/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5795 - loss: 0.7274 - val_accuracy: 0.6069 - val_loss: 0.6589
Epoch 6/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6040 - loss: 0.6739 - val_accuracy: 0.6069 - val_loss: 0.6545
Epoch 7/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6009 - loss: 0.6695 - val_accuracy: 0.5968 - val_loss: 0.6700
Epoch 8/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6145 - loss


Precisión final del modelo en datos de prueba: 0.6609
Modelo de audio guardado exitosamente en: /content/drive/MyDrive/audio_emotion_model.h5
Puedes descargarlo desde tu Google Drive y colocarlo en la carpeta 'model/' de tu proyecto Streamlit.
